In [1]:
import json
from pathlib import Path

import pandas as pd
from tensorflow import keras

from generator import Generator

In [2]:
MODEL = "/app/_data/work/models/A1_frozen_aug_amp"

In [46]:
df = pd.read_csv("/app/_data/work/work.csv")
model = keras.models.load_model(f"{MODEL}.h5")
meta = json.loads(Path(f"{MODEL}.meta.json").read_text())

In [45]:
def _y_to_labels(y):
    labels = []

    for i, v in enumerate(y):
        if v > 0.5:
            labels.append(meta["classes"][i])

    if len(labels) == 0:
        labels = ["_none_"]

    return " ".join(labels)

In [5]:
def normalize_labels(labels):
    return list(map(lambda x: " ".join(sorted(x.lower().split(" "))), labels))

In [6]:
g = Generator(
    df,
    shuffle=False,
    batch_size=1,
    image_output_size=meta["image_size"][:2],
    augmentation_options=None,
)

In [7]:
Y_pred = model.predict(g, verbose=1)

18568/18568 [==============================] - 268s 14ms/step


In [47]:
labels_pred = list(map(_y_to_labels, Y_pred))
labels_pred = normalize_labels(labels_pred)

In [48]:
df.labels = normalize_labels(list(df.labels))
df.insert(df.shape[1], "labels_predicted", labels_pred)
df.to_csv("eval.csv", index=False)

In [49]:
df_err = df[df.labels != df.labels_predicted]
df_err.to_csv("errors.csv", index=False)
df_err.shape[0]

1371

In [50]:
df_err.groupby('labels').count()

,image,Y,fold,labels_predicted
labels,,,,
complex,374,374,374,374
complex frog_eye_leaf_spot,115,115,115,115
complex frog_eye_leaf_spot scab,136,136,136,136
complex powdery_mildew,23,23,23,23
complex rust,36,36,36,36
frog_eye_leaf_spot,93,93,93,93
frog_eye_leaf_spot rust,87,87,87,87
frog_eye_leaf_spot scab,277,277,277,277
healthy,25,25,25,25


In [51]:
df_err[df_err.labels_predicted == "_none_"].groupby('labels').count()

,image,Y,fold,labels_predicted
labels,,,,
complex,50,50,50,50
frog_eye_leaf_spot,12,12,12,12
frog_eye_leaf_spot rust,2,2,2,2
frog_eye_leaf_spot scab,7,7,7,7
healthy,6,6,6,6
powdery_mildew,4,4,4,4
rust,12,12,12,12
scab,27,27,27,27
